<a href="https://colab.research.google.com/github/AAhmadS/deeplearning-HW4/blob/main/After_full_run_NLP_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
%cd /content/drive/MyDrive/deep learning

/content/drive/MyDrive/deep learning


In [30]:
!pip install torchtext==0.5.0
!git clone https://github.com/amnghd/Persian_poems_corpus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'Persian_poems_corpus' already exists and is not an empty directory.


In [31]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

In [32]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections
import torch

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#Section A :
##loading data

In [34]:
poets = ['khajoo','asad','bahar','feyz','asadi','hafez','jami','kamal','moulavi','parvin']
poems_data = []
label = []

for i,p in enumerate(poets):
  data = open('Persian_poems_corpus/normalized/'+p+'_norm.txt').read()
  poems = data.lower().split("\n")[1:]
  for poem in poems:
    if'  \t' != poem :
      poems_data.append(poem)
      label.append(i)
poems = pd.DataFrame({"poem": poems_data,
                     "poet": label})
poems.to_csv('poems.csv')

In [35]:
poems.head(5)

,poem,poet
0,ای غره ماه از اثر صنع تو غرا,0
1,وی طره شب از دم لطف تو مطرا,0
2,نوک قلم صنع تودر مبدا فطرت,0
3,انگیخته برصفحه کن صورت اشیا,0
4,سجاده نشینان نه ایوان فلک را,0


In [36]:
poems['poem_len_by_words'] = poems['poem'].apply(lambda t: len(hazm.word_tokenize(t)))

min_max_len = poems["poem_len_by_words"].min(), poems["poem_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 0 	Max: 15


In [37]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='poem_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [38]:
data_gl_than(poems, 15, 3)

Texts with word length of greater than 3 and less than 15 includes 86.67% of the whole!


In [39]:
poems['poem_len_by_words'] = poems['poem_len_by_words'].apply(lambda len_t: len_t if 3 < len_t <= 15 else None)
poems = poems.dropna(subset=['poem_len_by_words'])
poems = poems.reset_index(drop=True)

In [40]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=poems['poem_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within poems',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [41]:
fig = go.Figure()

groupby_rate = poems.groupby('poet')['poet'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of poets within poems',
    xaxis_title_text='poet',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

##Balancing

In [42]:
data0 = poems[poems['poet'] == 0]
data1 = poems[poems['poet'] == 1]
data2 = poems[poems['poet'] == 2]
data3 = poems[poems['poet'] == 3]
data4 = poems[poems['poet'] == 4]
data5 = poems[poems['poet'] == 5]
data6 = poems[poems['poet'] == 6]
data7 = poems[poems['poet'] == 7]
data8 = poems[poems['poet'] == 8]
data9 = poems[poems['poet'] == 9]

cutting_point = 9430

if cutting_point <= len(data0):
    data0 = data0.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data1):
    data1 = data1.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data2):
    data2 = data2.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data3):
    data3 = data3.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data4):
    data4 = data4.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data5):
    data5 = data5.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data6):
    data6 = data6.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data7):
    data7 = data7.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data8):
    data8 = data8.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data9):
    data9 = data9.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([data0,data1,data2,data3,data4,data5,data6,data7,data8,data9])
new_data = new_data.sample(frac=1).reset_index(drop=True)

In [43]:
fig = go.Figure()

groupby_label = new_data.groupby('poet')['poet'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of poets within poems [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [44]:
new_data.head(5)

,poem,poet,poem_len_by_words
0,دین ندارد بوی مشک و عود نیست,8,7.0
1,ازجان ودل نباشدچون بنده دوستدارت,7,5.0
2,شبگیر ار بگذرد نسیم بر آن زلف,2,7.0
3,ستارگان بتناوب شوند چهره گشای,7,5.0
4,چشم زخم دهر از ایشان دور بود,6,7.0


##train_test_val

In [45]:
train, test = train_test_split(new_data, test_size=0.2, random_state=1, stratify=new_data['poet'])
train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['poet'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['poem'].values.tolist(), train['poet'].values.tolist()
x_valid, y_valid = valid['poem'].values.tolist(), valid['poet'].values.tolist()
x_test, y_test = test['poem'].values.tolist(), test['poet'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(60352, 3)
(15088, 3)
(18860, 3)


#Section B:
##task a

In [46]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

### Configuration

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [48]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [49]:
# create a key finder based on label 2 id and id to label

label2id = {label: i for i, label in enumerate(poets)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'khajoo': 0, 'asad': 1, 'bahar': 2, 'feyz': 3, 'asadi': 4, 'hafez': 5, 'jami': 6, 'kamal': 7, 'moulavi': 8, 'parvin': 9}
id2label: {0: 'khajoo', 1: 'asad', 2: 'bahar', 3: 'feyz', 4: 'asadi', 5: 'hafez', 6: 'jami', 7: 'kamal', 8: 'moulavi', 9: 'parvin'}


In [50]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "khajoo",
    "1": "asad",
    "2": "bahar",
    "3": "feyz",
    "4": "asadi",
    "5": "hafez",
    "6": "jami",
    "7": "kamal",
    "8": "moulavi",
    "9": "parvin"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "asad": 1,
    "asadi": 4,
    "bahar": 2,
    "feyz": 3,
    "hafez": 5,
    "jami": 6,
    "kamal": 7,
    "khajoo": 0,
    "moulavi": 8,
    "parvin": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



### Input Embeddings

In [51]:
idx = np.random.randint(0, len(train))
sample_poem = train.iloc[idx]['poem']
sample_poet = train.iloc[idx]['poet']

print(f'Sample: \n{sample_poem}\n{sample_poet}')

Sample: 
و آنک مردن پیش او شد فتح باب
8


In [52]:
tokens = tokenizer.tokenize(sample_poem)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  poem: {sample_poem}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  poem: و آنک مردن پیش او شد فتح باب
  Coded_tokens: ['و', 'انک', 'مردن', 'پیش', 'او', 'شد', 'فتح', 'باب']
   Tokens: و انک مردن پیش او شد فتح باب
Token IDs: [1379, 24873, 22574, 2958, 2866, 2817, 6413, 5338]


In [53]:
encoding = tokenizer.encode_plus(
    sample_poem,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  1379, 24873, 22574,  2958,  2866,  2817,  6413,  5338,     4,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


### Dataset

In [54]:
class PoemsDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, poems, targets=None, label_list=None, max_len=128):
        self.poems = poems
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.poems)

    def __getitem__(self, item):
        poem = str(self.poems[item])

        if self.has_target:
            target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            poem,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'poem': poem,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = PoemsDataset(
        poems=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [55]:
label_list = poets
train_data_loader = create_data_loader(train['poem'].to_numpy(), train['poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['poem'].to_numpy(), valid['poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [56]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['poem'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'])

dict_keys(['poem', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['ز گرد دو رنگ اسپ ایشان به راه', 'رخت ما پیش از نزول ما بمنزل برده اند', 'چشم تو که بوسه گاه خلق بود', 'هنجار خطیر تلخ کامی را', 'هر که خواهد از تو از یک تا هزار', 'گر جهان طوفان غم گیرد چه باک', 'اهل نظر دو عالم در یک نظر ببازند', 'همان بهتر که من خالی کنم جای', 'یوسف مصر را بجان عزیز', 'نموده غنچه گل خنده آهنگ', 'گر تو باشی تنگ دل از ملحمه', 'جان می نتوان فدای آن کرد', 'من برخی شبم که یکی پرده افکند', 'ز دینار گنجی چهل چرم گاو', 'چو روزی بگذرد بر ما ز گیهان', 'طبع ما خرم از اندیشه اوست']
torch.Size([16, 128])
tensor([   2, 1363, 3221, 2897, 3880, 4085, 4460, 2789, 3148,    4,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
  

In [57]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['poem', 'input_ids', 'attention_mask', 'token_type_ids'])


In [58]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 10)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)['pooler_output']
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [59]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [60]:
pt_model = ClassifierModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


In [61]:
for name,param in pt_model.named_parameters():
  if'classifier' not in name:
    param.requires_grad = False
  else:
    print(name)

classifier.weight
classifier.bias


### Training

In [62]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb

In [39]:
from torch import optim

optimizer = optim.Adam(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1000...Train Loss: 2.314770...Train Acc: 0.101...Valid Loss: 2.292281...Valid Acc: 0.123...
Validation loss decreased (inf --> 2.292281).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 3/3...Step: 10000...Train Loss: 2.223802...Train Acc: 0.200...Valid Loss: 2.216829...Valid Acc: 0.202...
Validation loss decreased (2.218873 --> 2.216829).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 3/3...Step: 11000...Train Loss: 2.222285...Train Acc: 0.202...Valid Loss: 2.215928...Valid Acc: 0.202...
Validation loss decreased (2.216829 --> 2.215928).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

### Prediction

In [67]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [41]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [42]:
preds

array([1, 4, 1, ..., 2, 2, 1])

In [43]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['asad', 'asadi', 'asad', 'feyz', 'asadi']

##F1 Score and report

In [44]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.18851933257584408
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.19      0.22      0.21      1886
        asad       0.18      0.23      0.20      1886
       bahar       0.19      0.16      0.17      1886
        feyz       0.20      0.29      0.24      1886
       asadi       0.24      0.47      0.32      1886
       hafez       0.22      0.16      0.18      1886
        jami       0.17      0.10      0.13      1886
       kamal       0.17      0.06      0.09      1886
     moulavi       0.18      0.18      0.18      1886
      parvin       0.21      0.14      0.17      1886

    accuracy                           0.20     18860
   macro avg       0.20      0.20      0.19     18860
weighted avg       0.20      0.20      0.19     18860



In [45]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[415 225  93 295 254 199  88  78 166  73]
 [142 441 136 219 506  80  78  52 105 127]
 [145 207 306 179 382  98 148  63 207 151]
 [270 209  92 542 132 154  81  61 228 117]
 [ 92 258 182  92 888  35  99  42  96 102]
 [318 193 129 315 192 296 110  56 161 116]
 [168 222 191 252 403 104 193  73 194  86]
 [239 244 157 318 277 144 124 113 174  96]
 [157 216 168 238 335 110 114  81 334 133]
 [204 252 170 216 337 115 107  55 166 264]]


#Section C:
##fine-tuning the model:

##Adam

In [46]:
pt_ft_model = ClassifierModel(config=config)
pt_ft_model = pt_ft_model.to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


###Training

In [47]:
optimizer = optim.Adam(pt_ft_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_ft_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_ft_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1000...Train Loss: 1.941747...Train Acc: 0.291...Valid Loss: 1.719793...Valid Acc: 0.394...
Validation loss decreased (inf --> 1.719793).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 3/3...Step: 10000...Train Loss: 0.370710...Train Acc: 0.885...Valid Loss: 1.561515...Valid Acc: 0.560...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 3/3...Step: 11000...Train Loss: 0.311086...Train Acc: 0.907...Valid Loss: 1.562173...Valid Acc: 0.563...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

###Prediction

In [48]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_ft_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [49]:
preds

array([4, 9, 5, ..., 0, 9, 6])

In [50]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['asadi', 'parvin', 'hafez', 'hafez', 'asadi']

###F1 Score and report



In [51]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.5605413589060535
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.58      0.53      0.56      1886
        asad       0.71      0.72      0.71      1886
       bahar       0.40      0.42      0.41      1886
        feyz       0.56      0.55      0.55      1886
       asadi       0.72      0.73      0.72      1886
       hafez       0.56      0.58      0.57      1886
        jami       0.53      0.56      0.54      1886
       kamal       0.46      0.43      0.44      1886
     moulavi       0.56      0.53      0.54      1886
      parvin       0.53      0.55      0.54      1886

    accuracy                           0.56     18860
   macro avg       0.56      0.56      0.56     18860
weighted avg       0.56      0.56      0.56     18860



In [52]:
print(confusion_matrix(y_test,y_pred))

[[1006   15   74  151    7  278   35  191   58   71]
 [  15 1360   68   22  195    9   81   43   23   70]
 [  58   61  800   85  116  125  174  169  148  150]
 [ 138   30   77 1038    9  137   87  139  115  116]
 [   7  215   93    7 1382    5   71   17   21   68]
 [ 224    9  154  126    5 1098   39  110   54   67]
 [  30   93  182   65  101   38 1053   80  130  114]
 [ 150   43  179  151   35  126  143  807  133  119]
 [  51   32  175  123   22   52  173  121 1002  135]
 [  46   69  195   98   56   78  127   76  109 1032]]


##SGD

###Training

In [64]:
pt_ft_sgd_model = ClassifierModel(config=config)
pt_ft_sgd_model = pt_ft_sgd_model.to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
optimizer = optim.SGD(pt_ft_sgd_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_ft_sgd_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_ft_sgd_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1000...Train Loss: 2.338032...Train Acc: 0.101...Valid Loss: 2.313577...Valid Acc: 0.111...
Validation loss decreased (inf --> 2.313577).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 3/3...Step: 10000...Train Loss: 2.289413...Train Acc: 0.132...Valid Loss: 2.287210...Valid Acc: 0.135...
Validation loss decreased (2.287640 --> 2.287210).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 3/3...Step: 11000...Train Loss: 2.289140...Train Acc: 0.131...Valid Loss: 2.287025...Valid Acc: 0.135...
Validation loss decreased (2.287210 --> 2.287025).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

###Prediction

In [68]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_ft_sgd_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [69]:
preds

array([2, 9, 9, ..., 3, 5, 8])

In [70]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['bahar', 'parvin', 'parvin', 'moulavi', 'bahar']

###F1 Score and report



In [71]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.12777285187088389
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.12      0.18      0.15      1886
        asad       0.13      0.15      0.14      1886
       bahar       0.12      0.12      0.12      1886
        feyz       0.12      0.13      0.13      1886
       asadi       0.16      0.17      0.16      1886
       hafez       0.13      0.11      0.12      1886
        jami       0.14      0.10      0.11      1886
       kamal       0.12      0.17      0.14      1886
     moulavi       0.11      0.07      0.09      1886
      parvin       0.15      0.10      0.12      1886

    accuracy                           0.13     18860
   macro avg       0.13      0.13      0.13     18860
weighted avg       0.13      0.13      0.13     18860



In [74]:
print(confusion_matrix(y_test,y_pred))

[[345 226 109 297 167 159  78 278 102 125]
 [308 283 252 172 212 164 116 147 104 128]
 [232 200 235 129 228 138 174 290 158 102]
 [322 203 108 240 169 105  97 412  90 140]
 [229 203 356  95 318 124 168 141 138 114]
 [303 218 133 259 158 199 100 240 117 159]
 [243 199 213 188 206 128 182 271 153 103]
 [259 251 170 202 164 146 138 313 122 121]
 [226 205 223 205 182 121 150 344 134  96]
 [318 199 175 134 166 197 134 232 134 197]]


##Result:
####With almost the same amount of training time the Adam optimized model has done a much better job, It seems like that the sgd couldn't handle the difficulty differences of texts(Although it is hard to judge with this state of training). Also, fine-tuned models seem to be able to do much better jobs than previously checked model. 

#Source:
##Code was extracted and used from : https://colab.research.google.com/github/hooshvare/parsbert/blob/master/notebooks/Taaghche_Sentiment_Analysis.ipynb, just a little bit change was needed during the process.
